In [8]:
import matplotlib.pyplot as plt
from PyEMD import EMD
import os
from scipy import signal
from scipy import interpolate
from scipy import signal
import numpy as np
import math
import cmath
import pandas as pd
import pickle
import time 
from os import listdir
from os.path import isfile,isdir, join

In [9]:
def create_directory(name):
    try:
        #Create target Directory
        os.mkdir(name)
        print("Directory ",name,"Created")
    except FileExistsError:
        print("Directory",name,"already exists")
    return name

def save_fig(x,y,color,title,dir_):
    plt.figure() #figsize=(10,20)
    plt.plot(x,y,color)
    plt.title(title)
    plt.savefig(dir_)
    plt.close()

In [10]:
def imf_create(t,data):
    mins = signal.argrelmin(data)[0]
    mins_ = [float(data*60/8064) for data in mins]
    maxs = signal.argrelmax(data)[0]
    maxs_ = [float(data*60/8064) for data in maxs]
    #extrema = np.concatenate((mins, maxs))
    spl_min = interpolate.CubicSpline(mins_, data[mins])#, bc_type = 'natural') #clamped
    #l_env = spl_min(t)
    
    spl_max = interpolate.CubicSpline(maxs_, data[maxs])#, bc_type = 'natural')#clamped
    #u_env = spl_max(t)
    mid = (spl_max(t)+spl_min(t))/2
    
    #plt.figure()
    #plt.plot(t,data)
    #plt.plot(t,l_env,'-')
    #plt.plot(t,u_env,'-')
    #plt.plot(t, mid, '--')
    #plt.title('Plottings')
    #plt.show()
    
    return data-mid

def stopping_conditions(imf,t):
    mins = signal.argrelmin(imf)[0]
    mins_ = [float(data*60/8064) for data in mins]
    maxs = signal.argrelmax(imf)[0]
    maxs_ = [float(data*60/8064) for data in maxs]
    
    spl_min = interpolate.CubicSpline(mins_,imf[mins])#, bc_type = 'natural') #clamped
    spl_max = interpolate.CubicSpline(maxs_, imf[maxs])#, bc_type = 'natural')#clamped
    
    mean_amplitude = [np.abs(spl_max(i)+spl_min(i))/2 for i in range(0,len(t))]
    envelope_amplitude = [np.abs(spl_max(i)- spl_min(i))/2 for i in range(0,len(t))]
    bo = [(m/e > 0.05) for m,e in zip(mean_amplitude,envelope_amplitude)]
    
    #at each point, mean_amplitude < THRESHOLD2*envelope_amplitude
    condition = [not(m < 0.5*e) for m,e in zip(mean_amplitude,envelope_amplitude)]
    
    #mean of boolean array {(mean_amplitude)/(envelope_amplitude) > THRESHOLD} < TOLERANCE
    if((1 in condition) or (not(np.mean(bo)<0.05))):
        return False
    
    # |#zeros-#extrema|<=1
    zero_crossings = np.where(np.diff(np.signbit(imf)))[0]
    diff_zeroCr_extremas = np.abs(len(maxs)+len(mins)-len(zero_crossings))
    if(diff_zeroCr_extremas <= 1):# and mean <0.1):
        return True
    else:
        return False

In [11]:
def calc_area_of_sodp(X,Y,i,channel):
    #Area of Second Order Difference Plot
    SX = math.sqrt(np.sum(np.multiply(X,X))/len(X))
    SY = math.sqrt(np.sum(np.multiply(Y,Y))/len(Y))
    SXY = np.sum(np.multiply(X,Y))/len(X)
    D = cmath.sqrt((SX*SX) + (SY*SY) - (4*(SX*SX*SY*SY - SXY*SXY)))
    a = 1.7321 *cmath.sqrt(SX*SX + SY*SY + D)
    b = 1.7321 * cmath.sqrt(SX*SX + SY*SY - D)
    Area = math.pi *a *b
    print("Channel=  ",channel,"Area of SODP of IMF number= ",i, " is ", Area)
    return Area

def calc_mean_and_ctm(X,Y,i,channel):
    features = pd.DataFrame(columns=['radius','mean_distance','central_tendency_measure'])
    r = 0.5
    d = [ math.sqrt(X[i]*X[i] + Y[i]*Y[i]) for i in range (0,len(X))]
    delta = [1 if i<r else 0 for i in d]
    d = [i for i in d if i<r]
        
    ctm = np.sum(delta[:-2])/(len(delta)-2)
    mean_distance = np.mean(d)
    
    features.loc[0] = [r] + [ctm] + [mean_distance]
    return features

In [12]:
def second_order_difference_plot(y, i, channel,dir_,imp_features,trial):
    #remove outliers
    upper_quartile = np.percentile(y,80)
    lower_quartile = np.percentile(y,20)
    IQR = (upper_quartile - lower_quartile) * 1.5
    quartileSet = (lower_quartile- IQR, upper_quartile +IQR)
    y = y[np.where((y >= quartileSet[0]) & (y <= quartileSet[1]))]
    
    #plotting SODP
    X = np.subtract(y[1:],y[0:-1]) #x(n+1)-x(n)
    Y = np.subtract(y[2:],y[0:-2]).tolist()#x(n+2)-x(n-1)
    Y.extend([0])
    save_fig(X,Y,'.','SODP'+str(i),dir_+'/SODP'+str(i)+'.png')
    
    Area = calc_area_of_sodp(X,Y,i,channel)
    features =calc_mean_and_ctm(X,Y,i,channel)
    
    df = pd.DataFrame({"Trial":trial,"Channel":channel,"SODP_No":i,"Area":Area,
                       "m(r=0.5)":features[features['radius']==0.5]['mean_distance'],
                       "ctm(r=0.5)":features[features['radius']==0.5]['central_tendency_measure']})
    imp_features = imp_features.append(df,ignore_index=True)
    return imp_features

In [13]:
def emd_algorithm(channel,s,t, dirc_,imp_features,trial):
    # Execute EMD on signal
    IMF = EMD().emd(s,t)
    N = IMF.shape[0]+1

    # Plot results
    plt.figure(figsize=(20,20))
    plt.subplot(N,1,1)
    plt.plot(t, s, 'r')
    plt.title("Input signal:")
    plt.xlabel("Time [s]")
    for n, imf in enumerate(IMF):
        if n==7:
            break
        plt.subplot(N,1,n+2)
        plt.plot(t, imf, 'g')
        plt.title("IMF "+str(n+1))
        plt.xlabel("Time [s]")
        imp_features = second_order_difference_plot(imf,n+1,channel,dirc_,imp_features,trial)

    plt.tight_layout()
    plt.savefig(dirc_+'/'+str(channel)+'.png')
    plt.close()
    return imp_features

In [14]:
path_src = "C:/Users/Dell/Desktop/EEG Arithmetic/eeg-during-mental-arithmetic-tasks-1.0.0/excel files/"
path_dest = "C:/Users/Dell/Desktop/EEG Arithmetic/"

onlyfiles = [d for d in listdir(path_src) if isfile(join(path_src, d))] 
for filename in onlyfiles:
    imp_features = pd.DataFrame(columns=['Trial','Channel','SODP_No','Area','m(r=0.5)','ctm(r=0.5)'])
    subject_no = int(filename[7:9])
    trial_no = int(filename[10:11])
    file = pd.read_excel(path_src+'/'+filename)
    if 'Unnamed: 0' in file.columns:
            file.drop(columns=['Unnamed: 0'], inplace= True)
    file.reset_index(drop = True,inplace = True)
    channels = file.columns
    t = np.linspace(0,60,len(file.index))
    
    curr_dir = create_directory(str(subject_no))
    dir_  = create_directory(curr_dir +'/Trial '+ str(trial_no))
    print("Currently in ", dir_," directory.")
    
    for channel in channels:
        s = file[channel].values
        imp_features = emd_algorithm(channel,s,t, dir_,imp_features,trial_no)

    writer = pd.ExcelWriter(dir_+'/Trial'+str(trial_no)+'.xlsx')
    imp_features.to_excel(writer, index=False)
    writer.save()

Directory  33 Created
Directory  33/Trial 2 Created
Currently in  33/Trial 2  directory.
Channel=   EEG Fp1 Area of SODP of IMF number=  1  is  (12.449676103060302+0j)
Channel=   EEG Fp1 Area of SODP of IMF number=  2  is  3.354065741441931j
Channel=   EEG Fp1 Area of SODP of IMF number=  3  is  4.5953472006808544j
Channel=   EEG Fp1 Area of SODP of IMF number=  4  is  2.8858334141551847j
Channel=   EEG Fp1 Area of SODP of IMF number=  5  is  1.3125246424514327j
Channel=   EEG Fp1 Area of SODP of IMF number=  6  is  0.5217796736516954j
Channel=   EEG Fp1 Area of SODP of IMF number=  7  is  0.06728584151902474j
Channel=   EEG Fp2 Area of SODP of IMF number=  1  is  (18.80156168169143+0j)
Channel=   EEG Fp2 Area of SODP of IMF number=  2  is  (2.8095813388237114+0j)
Channel=   EEG Fp2 Area of SODP of IMF number=  3  is  4.701398082556977j
Channel=   EEG Fp2 Area of SODP of IMF number=  4  is  3.1105466252538254j
Channel=   EEG Fp2 Area of SODP of IMF number=  5  is  1.686353592769257j
Ch

Channel=   EEG O2 Area of SODP of IMF number=  5  is  1.3905943151281348j
Channel=   EEG O2 Area of SODP of IMF number=  6  is  0.7659888039364295j
Channel=   EEG O2 Area of SODP of IMF number=  7  is  0.24872509320578234j
Channel=   EEG Fz Area of SODP of IMF number=  1  is  (13.144381888456188+0j)
Channel=   EEG Fz Area of SODP of IMF number=  2  is  (2.9004109875597037+0j)
Channel=   EEG Fz Area of SODP of IMF number=  3  is  4.707848029519499j
Channel=   EEG Fz Area of SODP of IMF number=  4  is  3.248612035835845j
Channel=   EEG Fz Area of SODP of IMF number=  5  is  1.2497540900465693j
Channel=   EEG Fz Area of SODP of IMF number=  6  is  0.3370499195403279j
Channel=   EEG Fz Area of SODP of IMF number=  7  is  0.09168810316668788j
Channel=   EEG Cz Area of SODP of IMF number=  1  is  (14.110022744695954+0j)
Channel=   EEG Cz Area of SODP of IMF number=  2  is  (5.290166399452382+0j)
Channel=   EEG Cz Area of SODP of IMF number=  3  is  4.6479974903707j
Channel=   EEG Cz Area of 

In [ ]:
path = "C:/Users/Dell/Desktop/EEG Arithmetic/eeg-during-mental-arithmetic-tasks-1.0.0/excel files/"
imp_features = pd.DataFrame(columns=['Trial','Channel','SODP_No','Area','m(r=0.5)','ctm(r=0.5)'])

channels =[1,2,3,4,7,11,12,14,15,16,17,18,19,20,21,24,25,29,30,32]   
t = np.linspace(0,60,8064) #Since mentioned the signal is for 60 seconds

curr_dir = create_directory(fname[-7:-4])
print("Currently in ", curr_dir," folder.")

start = time.clock()  

for trial in range (1,2):#(1,41)
    dir_  = create_directory(curr_dir +'/Trial '+ str(trial))
    print("Currently in ", dir_," directory.")
        
    for channel in channels:
        dirc_  = create_directory(dir_ +'/Channel '+ str(channel))
        print("Currently in ", dirc_," directory.")
        s = data[trial-1,channel-1,:]
        imp_features = emd_algorithm(channel,s,t, dirc_,imp_features,trial)

    writer = pd.ExcelWriter(dir_+'/Trial'+str(trial)+'.xlsx')
    imp_features.to_excel(writer, index=False)
    writer.save()
    imp_features.drop(imp_features.index, inplace=True)#empty dataframe for next iteration

elapsed = time.clock()   
print ("Time spent in function is: ", (elapsed-start)/60 , " mins")